In [180]:
import numpy as np
import pandas as pd
import time

# generate data

In [181]:
# In real-world scenarios, learning how the data was generated is impractical. Do not rely on this function while doing research.
def generate_data(dim, num):
    np.random.seed(1)
    x = np.random.normal(0, 10, [num, dim])
    np.random.seed(3)
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

# write your model class

In [182]:
# Modifying the overall structure is acceptable but not recommended
#SMO 算法
class SVM1:
    def __init__(self,C,tol,num_epoches=1000):
        """
        :param C:Upper Bound of Lagrange Multiplier C>0
        :param tol: tolerance of KKT condition  0<tol<1
        """
        #self.coef = np.zeros(num_features)
        #self.intercept = 0
        #self.alpha=np.zeros(num_samples)
        self.C=C
        self.tol=tol
        self.num_epoches=num_epoches
        
    def func(self,x):
        return np.dot(x,self.coef.T)+self.intercept
    
    def KKT_i(self,x_i,y_i,a_i):
        #if 满足KKT条件，返回True，0
        #else 返回False，np.abs(1-y_i*func(coef,intercept,x_i))
        if a_i<0:
            print(f'alpha_i={a_i}<0')
            print(f"ERROR:alpha_i={a_i}<0\n\n")
            return False,None
        elif a_i>=0 and a_i<=self.C:
            if np.abs(1-y_i*self.func(x_i))<self.tol:
                return True,0
            else:
                return False,np.abs(1-y_i*self.func(x_i))
        else:
            print(f'alpha_i={a_i}>{self.C}')
            print(f"ERROR:alpha_i={a_i}>{self.C}\n\n")
            return False,None
    def KKT(self,X,y):
        # X应为np.array(num_samples,num_features)
        # y应为np.array(num_samples)
        # alpha应为np.array(num_samples)
        # coef应为np.array(num_features)
        # intercept应为np.array(1)
        # if 每一个X[i,:],y[i,0],alpha[i]都满足KKT条件，返回True
        # else 返回False，违背KKT条件最严重的那个样本的index，其中违背的程度被记录在KKT_i的第二个返回值中
        #用state记录是否全部样本满足KKT条件
        STATE=True
        GAP=np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            state,gap=self.KKT_i(X[i,:],y[i],self.alpha[i])
            GAP[i]=gap
            if state==False:
                STATE=False
        #print(GAP)
        if STATE==True:
            return True,-1
        else:
            return False,np.random.choice(np.where(GAP==np.max(GAP))[0])
        
    def clipping(self,alpha,low,high):#注意这里要写self
        if alpha<low:
            return low
        elif alpha>high:
            return high
        else:
            return alpha
        
    def fit(self, X, y):
        num_features=X.shape[1]
        num_samples=X.shape[0]
        self.coef = np.zeros(num_features)
        self.intercept = 0
        self.alpha=np.zeros(num_samples)
        for epoch in range(self.num_epoches):
            #验证是否全部样本满足KKT条件
            state,index=self.KKT(X,y)
            if state==True:
                break
            else:
                i=index 
                #选择第二个样本
                E=np.zeros(X.shape[0])
                for k in range(X.shape[0]):
                    E[k]=self.func(X[k,:])-y[k]
                #j=np.argmax(np.abs(E-E[i]))
                j=np.random.choice(np.where(np.abs(E[i]-E)==np.max(np.abs(E[i]-E)))[0])
                assert j!=i,'Error: j==i, please check the code'
                #固定其他样本的alpha，更新alpha_i,alpha_j
                nita=1/(np.dot(X[i,:],X[i,:])+np.dot(X[j,:],X[j,:])-2*np.dot(X[i,:],X[j,:]))
                assert nita>0,'Error: nita<=0, please check the code'
                alpha_j_new_uncut=self.alpha[j]+y[j]*(E[i]-E[j])*nita
                #计算low和high
                if y[i]==y[j]:
                    low=max(0,self.alpha[i]+self.alpha[j]-self.C)
                    high=min(self.C,self.alpha[i]+self.alpha[j])
                else:
                    low=max(0,self.alpha[j]-self.alpha[i])
                    high=min(self.C,self.C+self.alpha[j]-self.alpha[i])
                #修剪alpha_j_new
                alpha_j_new=self.clipping(alpha_j_new_uncut,low,high)#报错？说是我放进去了4个参数，但是只有3个参数。后来发现是因为我把self放进去了,clipping函数一开始没写self
                alpha_i_new=self.alpha[i]+y[i]*y[j]*(self.alpha[j]-alpha_j_new)
                #更新coef
                #coef_new=coef+y[i]*(alpha_i_new-alpha[i])*X[i,:]+y[j]*(alpha_j_new-alpha[j])*X[j,:]
                #更新intercept
                b_i_new=self.intercept-E[i]-y[i]*(alpha_i_new-self.alpha[i])*np.dot(X[i,:],X[i,:])-y[j]*(alpha_j_new-self.alpha[j])*np.dot(X[i,:],X[j,:])
                b_j_new=self.intercept-E[j]-y[i]*(alpha_i_new-self.alpha[i])*np.dot(X[i,:],X[j,:])-y[j]*(alpha_j_new-self.alpha[j])*np.dot(X[j,:],X[j,:])
                #选择新的intercept
                if alpha_i_new>0 and alpha_i_new<self.C:
                    intercept_new=b_i_new
                elif alpha_j_new>0 and alpha_j_new<self.C:
                    intercept_new=b_j_new
                else:
                    intercept_new=(b_i_new+b_j_new)/2
                #更新alpha
                self.alpha[i]=alpha_i_new
                self.alpha[j]=alpha_j_new
                #更新coef
                #coef=coef_new
                #更新intercept
                self.intercept=intercept_new
                #print(f'epoch={epoch},i={i},j={j},\nalpha_{i}_new={alpha_i_new},alpha_{j}_new={alpha_j_new},intercept={self.intercept}\n\n')
        
        #更新coef:
        self.coef=np.zeros(X.shape[1])
        for i in range(X.shape[0]):
            self.coef+=self.alpha[i]*y[i]*X[i,:]
        print(f'self.coef={self.coef}')
        print(f'self.intercept={self.intercept}')
        
        
    def predict(self, X, y,Train=False):
        if Train==True:
            str='train set'
        elif Train==False:
            str='test set'
        else:
            print('ERROR:Train should be True or False')
        y_pred = np.sign(self.func(X))
        #correct rate
        print(f'{str} accuracy:\n{np.sum([y * y_pred >= 0])/len(X)}')
        print(f'infact mislabeled num:{np.sum([y * y_pred < 0])},{str} num:{len(X)}')
        #返回数据框
        return pd.DataFrame({'y_pred':y_pred,'y':y})

In [183]:
# Gradient Descent 算法
class SVM2:
    def __init__(self,num_epoches=150,lr=0.001,decay=0.05):
        self.num_epoches=num_epoches
        self.lr=lr
        self.decay=decay
        
    def func(self,x):
        return np.dot(x,self.coef.T)+self.intercept
    
    def target_function(self,X,y):
        return np.sum(self.alpha)-0.5*np.dot(np.dot(self.alpha*y,X),np.dot(self.alpha*y,X).T)
    
    def fit(self, X, y):
        target=0
        num_features=X.shape[1]
        num_samples=X.shape[0]
        self.coef = np.zeros(num_features)
        self.intercept = 0
        self.alpha = np.zeros(num_samples)
        for epoch in range(self.num_epoches):
            pre_target=target
            grad_alpha=np.ones(X.shape[0])-np.dot(np.dot(self.alpha*y,X),X.T)*y
            grad_alpha=grad_alpha.reshape(-1,1)
            self.alpha+=self.lr*grad_alpha.reshape(-1,)
            self.coef=np.dot(X.T,self.alpha*y)
            self.intercept=np.sum(self.alpha*y)
            target=self.target_function(X,y)
            if target<pre_target:
                self.lr*=0.05
            print('epoch: {}, target: {}'.format(epoch, target))
            if self.lr<1e-7:
                break
        
    def predict(self, X, y,Train=False):
        if Train==True:
            str='train set'
        elif Train==False:
            str='test set'
        else:
            print('ERROR:Train should be True or False')
        y_pred = np.sign(self.func(X))
        #correct rate
        print(f'{str} accuracy:\n{np.sum([y * y_pred >= 0])/len(X)}')
        print(f'infact mislabeled num:{np.sum([y * y_pred < 0])},{str} num:{len(X)}')
        #返回数据框
        return pd.DataFrame({'y_pred':y_pred,'y':y})

# construct and train your models

In [184]:
# generate data
num_features, num_samples = 20, 10000
split_rate = 0.8
X, y, mislabel = generate_data(dim=num_features, num=num_samples)
print(f'mislabel rate:{mislabel}')
y = y.reshape(-1, )
X_std = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
X_train, y_train = X[:int(split_rate * len(y))], y[:int(split_rate * len(y))]
X_test, y_test = X[int(split_rate * len(y)):], y[int(split_rate * len(y)):]
X_train_std, X_test_std = X_std[:int(split_rate * len(y))], X_std[int(split_rate * len(y)):]
print(f'X_train.shape:{X_train.shape},y_train.shape:{y_train.shape}')
print(f'X_test.shape:{X_test.shape},y_test.shape:{y_test.shape}')

mislabel rate:0.0362
X_train.shape:(8000, 20),y_train.shape:(8000,)
X_test.shape:(2000, 20),y_test.shape:(2000,)


In [185]:
model1 = SVM1(C=5,tol=1e-4,num_epoches=500)
start_time=time.time()
model1.fit(X_train,y_train)
end_time=time.time()
duration_smo=end_time-start_time
print(f'训练过程用时:{duration_smo:.2f}s')

self.coef=[ 0.01719161  0.49146618 -0.89159247 -0.04808921  1.31688291  1.2633268
 -1.23353559 -0.96306783 -1.5568798  -0.27924405 -1.63717165 -0.13348756
  0.57195046 -0.89801116  0.3423029   0.50739982 -1.18616736  0.2674257
 -0.83847034 -0.30212554]
self.intercept=0.1324866365912034
训练过程用时:45.18s


In [186]:
model2 = SVM2(num_epoches=150,lr=0.0005,decay=0.1)
start_time=time.time()
model2.fit(X_train, y_train)
end_time=time.time()
duration_gd=end_time-start_time
print(f'训练过程用时:{duration_gd:.2f}s')

epoch: 0, target: -475.17183033489397
epoch: 1, target: -169108.38294729317
epoch: 2, target: -566.6064057916248
epoch: 3, target: -1.2270783373822658
epoch: 4, target: 1.6469084044141187
epoch: 5, target: 1.6680218158907443
epoch: 6, target: 1.6720604690925804
epoch: 7, target: 1.6759882191876547
epoch: 8, target: 1.679915215720204
epoch: 9, target: 1.6838422069831038
epoch: 10, target: 1.6877691982083944
epoch: 11, target: 1.6916961894334122
epoch: 12, target: 1.6956231806584279
epoch: 13, target: 1.6995501718834438
epoch: 14, target: 1.7034771631084595
epoch: 15, target: 1.7074041543334757
epoch: 16, target: 1.7113311455584912
epoch: 17, target: 1.7152581367835071
epoch: 18, target: 1.7191851280085229
epoch: 19, target: 1.7231121192335386
epoch: 20, target: 1.7270391104585545
epoch: 21, target: 1.7309661016835702
epoch: 22, target: 1.734893092908586
epoch: 23, target: 1.738820084133602
epoch: 24, target: 1.7427470753586176
epoch: 25, target: 1.7466740665836333
epoch: 26, target: 1.7

# predict and compare your results

In [187]:
# make prediction
print('model1:SMO algorithm')
model1.predict(X_train, y_train, Train=True)
model1.predict(X_test, y_test)

model1:SMO algorithm
train set accuracy:
0.9205
infact mislabeled num:636,train set num:8000
test set accuracy:
0.922
infact mislabeled num:156,test set num:2000


,y_pred,y
0,-1.0,-1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,-1.0,-1.0
...,...,...
1995,-1.0,-1.0
1996,1.0,1.0
1997,1.0,1.0
1998,-1.0,-1.0


In [188]:
# make prediction
print('model2:Gradient Descent algorithm')
model2.predict(X_train, y_train, Train=True)
model2.predict(X_test, y_test)

model2:Gradient Descent algorithm
train set accuracy:
0.94825
infact mislabeled num:414,train set num:8000
test set accuracy:
0.9385
infact mislabeled num:123,test set num:2000


,y_pred,y
0,-1.0,-1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,-1.0,-1.0
...,...,...
1995,-1.0,-1.0
1996,1.0,1.0
1997,1.0,1.0
1998,-1.0,-1.0


In [189]:
#compare with sklearn,并计算时间
start_time=time.time()
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
end_time=time.time()
print(f'clf.coef_={clf.coef_}')
print(f'clf.intercept_={clf.intercept_}')
df_support_vectors=pd.DataFrame(clf.support_vectors_.tolist())
print(f'testset accuracy:{clf.score(X_test,y_test)}')
duration_sklearn=end_time-start_time
print(f'用时:{duration_sklearn}:.2f')

clf.coef_=[[ 0.02274071  0.09404098 -0.1064336   0.00202636  0.18287085  0.1882266
  -0.17106342 -0.1420816  -0.2126855  -0.03034664 -0.22178982 -0.01407329
   0.06690528 -0.10493372  0.08157887  0.04348746 -0.22365544  0.01925659
  -0.11411043 -0.03441737]]
clf.intercept_=[-9.99186188e-05]
testset accuracy:0.951
用时:22.488031148910522


In [190]:
#打印出支持向量
df_support_vectors

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-2.223281,-2.007581,1.865614,4.100516,1.982997,1.190086,-6.706623,3.775638,1.218213,11.294839,11.989179,1.851564,-3.752850,-6.387304,4.234944,0.773401,-3.438537,0.435969,-6.200008,6.980320
1,-4.008782,8.240056,-5.623054,19.548781,-13.319517,-17.606886,-16.507213,-8.905556,-11.191154,19.560789,-3.264995,-13.426758,11.143830,-5.865239,-12.368534,8.758389,6.233622,-4.349567,14.075400,1.291016
2,0.722519,10.097873,-15.569416,-6.124421,-1.393518,-7.285375,5.311638,0.040008,3.212659,-7.252149,15.365363,-0.003750,12.935496,-4.389977,5.900395,-6.793838,-9.509093,-7.043503,-0.458667,-2.187335
3,-2.222195,9.083397,-1.586068,6.953060,-1.142183,-1.897672,12.591335,-7.519774,-2.830529,-12.927374,0.967394,10.695010,6.814019,7.340118,10.530424,6.252182,7.582594,4.037241,-9.753885,5.260951
4,0.663927,5.406022,-13.188968,8.454264,1.310922,3.490837,4.040681,5.124341,11.203618,8.604544,4.869104,-7.643396,2.863311,-5.578340,-14.487523,-0.413769,-9.128101,11.351414,-0.377092,0.452626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,-22.063162,22.288755,7.709771,-4.198827,-7.520452,4.956501,-6.177335,-0.373615,-15.021083,-1.099379,0.221363,9.919162,-6.048613,-10.875043,12.125939,-28.363086,12.978091,6.315078,11.285702,11.148314
1314,5.309228,6.522368,-5.596859,-6.032238,-8.548780,9.272599,15.140683,7.823205,-3.414320,12.725956,5.722150,13.983760,7.234502,-13.056546,-1.098291,-10.638276,-1.546051,-1.409231,-16.778285,-10.494791
1315,8.818781,-3.237233,-4.894439,2.398958,-12.368448,5.795863,12.562584,-12.341323,14.695609,18.516630,3.380304,3.061986,-2.469387,-7.355258,8.026687,4.560838,-17.343922,14.650620,-8.671666,3.911934
1316,16.874231,15.231831,4.707415,10.695622,2.737454,-11.643657,-4.609053,2.071136,1.920710,-5.010482,-3.073552,1.005002,-5.610403,-8.634869,-4.660314,-8.354574,1.415778,17.341982,-1.643440,0.613246
